In [7]:
import torch
import gymnasium as gym
import numpy as np
import mediapy as media
# 필요한 클래스 및 함수 임포트
from cleanrl.cleanrl.ppo_continuous_action import Agent, Args, ppo_make_env
import cv2
from cleanrl.cleanrl.ppo_continuous_action import NormalizeObservationCustom, NormalizeRewardCustom


def load_model_and_evaluate(model_path, task_id="HalfCheetah-v4", num_episodes=10, seed=1, gamma=0.99):
    """
    저장된 모델을 불러와 환경에서 평가를 수행하는 함수
    """
    visualize = True
    frames = []
    
    # Argument 설정
    args = Args()
    args.task_id = task_id
    args.seed = seed
    args.gamma = gamma
    args.fix_object = False
    control_mode = "osc_position"

    # 환경 생성
    env = gym.vector.SyncVectorEnv(
        [ppo_make_env(
            task_id=args.task_id, 
            reward_shaping=True,
            idx=0, 
            capture_video=False, 
            run_name="eval", 
            gamma= args.gamma, 
            active_rewards="rghl",
            #active_image=True, 
            fix_object=args.fix_object,
            wandb_enabled=False,
            control_mode=control_mode,
            )
        ]
    )
    
    
    # 디바이스 설정 (cuda가 가능하면 cuda 사용)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if device == torch.device("cuda"):
        print("Using CUDA")
    else :
        assert device == torch.device("cpu")


    weight_path = model_path + ".cleanrl_model"
    # Agent 초기화 및 모델 불러오기
    agent = Agent(env).to(device)
    agent.load_state_dict(torch.load(weight_path, map_location=device))
    agent.eval()  # 평가 모드로 전환
        
# 환경 생성 후
    env_instance = env.envs[0]
    # NormalizeObservationCustom 래퍼에 접근
    normalize_obs_wrapper = env_instance
    while not isinstance(normalize_obs_wrapper, NormalizeObservationCustom):
        normalize_obs_wrapper = normalize_obs_wrapper.env

    # NormalizeRewardCustom 래퍼에 접근
    normalize_reward_wrapper = env_instance
    while not isinstance(normalize_reward_wrapper, NormalizeRewardCustom):
        normalize_reward_wrapper = normalize_reward_wrapper.env

    # 상태 로드
    normalize_obs_wrapper.load_obs_running_average(model_path + "_obs_rms.npz")
    normalize_reward_wrapper.load_reward_running_average(model_path + "_reward_rms.npz")

    

    # 평가 수행
    total_rewards = []
    for episode in range(num_episodes):
        obs, _ = env.reset()
        obs = torch.Tensor(obs).to(device)
        done = False
        episode_reward = 0
        
        #image_frame = env.envs[0].image_states['agentview_image']
        #image_frame = np.array(image_frame[::-1, :, :], dtype=np.uint8)  # 명시적으로 numpy 배열로 변환
        #frames.append(image_frame)
        # convert image_frame cv2 image
        #image_frame = cv2.cvtColor(image_frame, cv2.COLOR_RGB2BGR)
                
        for i in range(300):
            with torch.no_grad():
                obs = torch.Tensor(obs).to(device)
                action, _, _, _ = agent.get_action_and_value(obs)
            obs, reward, terminations, truncations, infos = env.step(action.cpu().numpy())
            print(action.cpu().numpy())
            #print(f"reward: {reward}, terminations: {terminations}, truncations: {truncations}, infos: {infos}")
            done = np.logical_or(terminations, truncations).any()
            episode_reward += reward[0]  # 첫 번째 환경의 보상 합산
            
            # 새로운 프레임 가져오기 및 변환
            #image_frame = env.envs[0].image_states['agentview_image']
            #image_frame = np.array(image_frame[::-1, :, :], dtype=np.uint8)  # numpy 배열로 변환

            #image_frame = cv2.cvtColor(image_frame, cv2.COLOR_RGB2BGR)
            # draw text on image_frame episode reward, reward, small text
            #cv2.putText(image_frame, f"Episode Reward: {episode_reward:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            #cv2.putText(image_frame, f"Reward: {reward[0]:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            #frames.append(image_frame)
            
            
        print(f"Episode {episode + 1}: Total Reward: {episode_reward}")
        total_rewards.append(episode_reward)


    env.close()

    # 평균 리턴 출력
    avg_reward = np.mean(total_rewards)
    print(f"Average Reward over {num_episodes} episodes: {avg_reward}")
    
    #media.show_video(frames, fps=30)



In [8]:
# 저장된 모델 불러와 평가하기
#model_path = "/research/rs4tmr/cleanrl/cleanrl/runs/tr__ppo_continuous_action__s1__2024-09-26_05-11-00/ppo_continuous_action_380928.cleanrl_model"
model_path = "/research/rs4tmr/cleanrl/cleanrl/runs/lift_None_s1__2024-09-27 16:22:32/ppo_continuous_action_901120"#.cleanrl_model"  # 모델 경로 지정
load_model_and_evaluate(model_path, task_id="lift", num_episodes=1)

Initalized env with init_env
Lift
########################
### Observation keys ###
Key: robot0_eef_pos, size: 3
Key: robot0_eef_quat, size: 4
Key: robot0_eef_vel_lin, size: 3
Key: robot0_eef_vel_ang, size: 3
Key: robot0_gripper_qpos, size: 6
Key: robot0_gripper_qvel, size: 6
Key: gripper_to_cube_pos, size: 3
Key: robot0_proprio-state, size: 25
Key: object-state, size: 3
Total observation size: 56
########################
####### Options ########
task_id: lift
active_rewards: rghl
control_mode: OSC_POSITION
reward_shaping: True
fix_object: False
active_image: False
wandb_enabled: False
########################
Using CUDA
Total parameters: 12361


/tmp/ipykernel_27820/904149572.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent.load_state_dict(torch.load(model_path, map_location=device))


[[-0.02796695 -1.477859   -0.05040339 -0.20111877]]
[[-1.0724443  -0.3003863   0.51074755  1.540766  ]]
[[-0.565637    2.1040502  -1.5207204   0.68316215]]
[[ 1.4967408  -1.7946006   0.12783462 -2.165857  ]]
[[0.1518383  1.2755486  0.12471426 1.642849  ]]
[[-2.3843489  -0.5705758  -0.13163692 -0.0731647 ]]
[[-0.78059924  2.3325608   0.8370768   0.20188433]]
[[ 0.9655345  -1.3902311  -0.23632568 -1.0163846 ]]
[[ 0.9826178  -0.0634695  -0.23814464 -0.47681493]]
[[0.9141336 3.1866703 1.2639124 1.4340043]]
[[-1.5603963   1.3596854  -0.39610505  0.24920847]]
[[ 1.7522739 -3.5972118 -1.3740194  0.4880645]]
[[-1.7736858  -3.2727566  -1.990366   -0.04137397]]
[[-2.3580856  3.2200637 -1.1301103  1.0290623]]
[[-2.399599    3.0856586  -0.51348466  0.27728832]]
[[-2.1353784  1.3981187 -1.1156716 -1.7555425]]
[[-0.17289987 -1.2113864   1.3809332  -0.30835098]]
[[ 0.21271917  1.0748838   0.2854117  -0.57411695]]
[[2.1134107  1.3537916  1.4509022  0.31407544]]
[[ 1.3876524 -3.9765997 -1.2552077 -0.24